# EDA - Identify significant predictors with RF feature selection
- Edited by Rumi Nakagawa
- Spring 2023 Capstone

## SITE US-vcp
- ENF  Evergreen Needleleaf Forests
- metadata

|     | site_id |   dataset | start_year | end_year |                                              file | is_dup | IGBP | elevation |      lat |       long |
|----:|--------:|----------:|-----------:|---------:|--------------------------------------------------:|-------:|-----:|----------:|---------:|-----------:|
| 206 |  US-Vcp | AmeriFlux |       2007 |     2017 | FLX_US-Vcp_FLUXNET2015_FULLSET_MM_2007-2017_be... |  False |  ENF |    2542.0 | 35.86240 | -106.59740 |

# 0. Preparation

## Mount google drive
- Make sure that available access is the user's own drive(no access across files in shared folder)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd drive/MyDrive/

/content/drive/MyDrive


## Import libraries

In [3]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 19.7 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=9201a176e4d2bdde61c2c74f6e915c589bc5f91d9f0e63484b7e037c3a93b43c
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [4]:
# !pip install dython

In [5]:
!pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 KB 10.4 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 7.1.2
    Uninstalling click-7.1.2:
      Successfully uninstalled click-7.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 1.1.4 requires click<8.0,>=5.1, but you have click 8.1.3 which is incompatible.


In [6]:
# Import SparkSession
from pyspark.sql import SparkSession
# Create a Spark Session
spark = SparkSession.builder.master("local[*]").getOrCreate()
# Check Spark Session Information
spark

In [7]:
# Import a Spark function from library
from pyspark.sql.functions import col

In [8]:
from pyspark.sql.functions import col
from pyspark.ml.classification import LogisticRegression
from pyspark.sql.functions import col
import pandas as pd
import numpy as np
import os

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import ast
from pyspark.sql.functions import desc

import geopandas as gpd
import folium

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import MinMaxScaler
from pyspark.mllib.evaluation import MulticlassMetrics, BinaryClassificationMetrics
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from pyspark.ml.classification import RandomForestClassifier
import time
# from dython import nominal

# Apply the default theme
sns.set_theme()


## (to be updated) Get access to blob storage
- Reference from 261

In [9]:
# Put at the top of any notebooks for storing in blob

# from pyspark.sql.functions import col, max

# blob_container = "team06" # The name of your container created in https://portal.azure.com
# storage_account = "apatel" # The name of your Storage account created in https://portal.azure.com
# secret_scope = "team06" # The name of the scope created in your local computer using the Databricks CLI
# secret_key = "team06" # The name of the secret key created in your local computer using the Databricks CLI 
# blob_url = f"wasbs://{blob_container}@{storage_account}.blob.core.windows.net"
# mount_path = "/mnt/mids-w261"

## Import csv
sample csv

In [10]:
us_vcp_hourly_df = pd.read_csv("data_full_half_hourly_raw_v0_1_US-Vcp_mydrive.csv")
# Copied original file. It is needed to store in each user's mydrive


In [11]:
us_vcp_hourly_df

,TIMESTAMP_START,TIMESTAMP_END,TA_F,TA_F_QC,TA_ERA,SW_IN_POT,SW_IN_F,SW_IN_F_QC,SW_IN_ERA,LW_IN_F,...,NIRv,b1,b2,b3,b4,b5,b6,b7,IGBP,koppen
0,200701010000,200701010030,-7.706,0,-9.299,0.0,0.0,0,0.000,202.327,...,0.110525,0.1631,0.3285,0.1270,0.1362,0.1932,0.0567,0.0283,ENF,Cold
1,200701010030,200701010100,-8.095,0,-9.672,0.0,0.0,0,0.000,202.327,...,0.110525,0.1631,0.3285,0.1270,0.1362,0.1932,0.0567,0.0283,ENF,Cold
2,200701010100,200701010130,-8.582,0,-10.044,0.0,0.0,0,0.000,202.327,...,0.110525,0.1631,0.3285,0.1270,0.1362,0.1932,0.0567,0.0283,ENF,Cold
3,200701010130,200701010200,-8.136,0,-10.417,0.0,0.0,0,0.000,202.327,...,0.110525,0.1631,0.3285,0.1270,0.1362,0.1932,0.0567,0.0283,ENF,Cold
4,200701010200,200701010230,-8.393,0,-10.789,0.0,0.0,0,0.000,208.388,...,0.110525,0.1631,0.3285,0.1270,0.1362,0.1932,0.0567,0.0283,ENF,Cold
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174523,201712312130,201712312200,-0.972,0,1.230,0.0,0.0,0,137.007,243.894,...,0.093599,0.0516,0.1731,0.0283,0.0422,0.1897,0.1439,0.0878,ENF,Cold
174524,201712312200,201712312230,-1.390,0,1.230,0.0,0.0,0,137.007,243.894,...,0.093599,0.0516,0.1731,0.0283,0.0422,0.1897,0.1439,0.0878,ENF,Cold
174525,201712312230,201712312300,-1.961,0,1.230,0.0,0.0,0,137.007,243.894,...,0.093599,0.0516,0.1731,0.0283,0.0422,0.1897,0.1439,0.0878,ENF,Cold
174526,201712312300,201712312330,-1.802,0,1.230,0.0,0.0,0,137.007,243.894,...,0.093599,0.0516,0.1731,0.0283,0.0422,0.1897,0.1439,0.0878,ENF,Cold


### Add column that explains start time
- whether its start of TIMESTAMP is "XX:00" or "XX:30"
- XX:00 -> A
- XX:30 -> B

In [12]:
len('200701010000')
x = '200701010000'
x[10:]

'00'

In [13]:
def minute_categorical(x):
  if x == '00':
    return 'A'
  if x == '30':
    return 'B'
  else: 
    return 'NA'

us_vcp_hourly_df['min_categorical'] = us_vcp_hourly_df['TIMESTAMP_START'].apply(lambda x: minute_categorical(str(x)[10:]))

In [14]:
us_vcp_hourly_df[['TIMESTAMP_START','min_categorical']]

,TIMESTAMP_START,min_categorical
0,200701010000,A
1,200701010030,B
2,200701010100,A
3,200701010130,B
4,200701010200,A
...,...,...
174523,201712312130,B
174524,201712312200,A
174525,201712312230,B
174526,201712312300,A


In [15]:
# Reduce the amount of dataset 
us_vcp_hourly_00_df = us_vcp_hourly_df[us_vcp_hourly_df['min_categorical'] == 'A']
us_vcp_hourly_00_df

,TIMESTAMP_START,TIMESTAMP_END,TA_F,TA_F_QC,TA_ERA,SW_IN_POT,SW_IN_F,SW_IN_F_QC,SW_IN_ERA,LW_IN_F,...,b1,b2,b3,b4,b5,b6,b7,IGBP,koppen,min_categorical
0,200701010000,200701010030,-7.706,0,-9.299,0.0,0.0,0,0.000,202.327,...,0.1631,0.3285,0.1270,0.1362,0.1932,0.0567,0.0283,ENF,Cold,A
2,200701010100,200701010130,-8.582,0,-10.044,0.0,0.0,0,0.000,202.327,...,0.1631,0.3285,0.1270,0.1362,0.1932,0.0567,0.0283,ENF,Cold,A
4,200701010200,200701010230,-8.393,0,-10.789,0.0,0.0,0,0.000,208.388,...,0.1631,0.3285,0.1270,0.1362,0.1932,0.0567,0.0283,ENF,Cold,A
6,200701010300,200701010330,-8.417,0,-10.989,0.0,0.0,0,0.000,208.388,...,0.1631,0.3285,0.1270,0.1362,0.1932,0.0567,0.0283,ENF,Cold,A
8,200701010400,200701010430,-8.239,0,-11.188,0.0,0.0,0,0.000,208.388,...,0.1631,0.3285,0.1270,0.1362,0.1932,0.0567,0.0283,ENF,Cold,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174518,201712311900,201712311930,1.235,0,1.230,0.0,0.0,0,137.007,243.894,...,0.0516,0.1731,0.0283,0.0422,0.1897,0.1439,0.0878,ENF,Cold,A
174520,201712312000,201712312030,0.711,0,1.230,0.0,0.0,0,137.007,243.894,...,0.0516,0.1731,0.0283,0.0422,0.1897,0.1439,0.0878,ENF,Cold,A
174522,201712312100,201712312130,-1.199,0,1.230,0.0,0.0,0,137.007,243.894,...,0.0516,0.1731,0.0283,0.0422,0.1897,0.1439,0.0878,ENF,Cold,A
174524,201712312200,201712312230,-1.390,0,1.230,0.0,0.0,0,137.007,243.894,...,0.0516,0.1731,0.0283,0.0422,0.1897,0.1439,0.0878,ENF,Cold,A


## Preprocess features

In [16]:
# Original columns
print(len(us_vcp_hourly_df.columns))
us_vcp_hourly_df.columns

65


Index(['TIMESTAMP_START', 'TIMESTAMP_END', 'TA_F', 'TA_F_QC', 'TA_ERA',
       'SW_IN_POT', 'SW_IN_F', 'SW_IN_F_QC', 'SW_IN_ERA', 'LW_IN_F',
       'LW_IN_F_QC', 'LW_IN_ERA', 'VPD_F', 'VPD_F_QC', 'VPD_ERA', 'P_F',
       'P_F_QC', 'P_ERA', 'PA_F', 'PA_F_QC', 'PA_ERA', 'NETRAD', 'PPFD_IN',
       'G_F_MDS', 'G_F_MDS_QC', 'LE_F_MDS', 'LE_F_MDS_QC', 'LE_CORR',
       'H_F_MDS', 'H_F_MDS_QC', 'H_CORR', 'NEE_VUT_REF', 'NEE_VUT_REF_QC',
       'NEE_CUT_REF', 'NEE_CUT_REF_QC', 'GPP_NT_VUT_REF', 'GPP_DT_VUT_REF',
       'GPP_NT_CUT_REF', 'GPP_DT_CUT_REF', 'RECO_NT_VUT_REF',
       'RECO_DT_VUT_REF', 'RECO_NT_CUT_REF', 'RECO_DT_CUT_REF', 'datetime',
       'year', 'month', 'day', 'hour', 'SITE_ID', 'date', 'NEE_VUT_REF_qa',
       'SW_DIF', 'EVI', 'NDVI', 'NIRv', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6',
       'b7', 'IGBP', 'koppen', 'min_categorical'],
      dtype='object')

## Check how much NA/None exists in each column

In [17]:
us_vcp_hourly_df.isna().sum()

TIMESTAMP_START        0
TIMESTAMP_END          0
TA_F                   0
TA_F_QC                0
TA_ERA                 0
                   ...  
b6                 17664
b7                 10896
IGBP                   0
koppen                 0
min_categorical        0
Length: 65, dtype: int64

In [18]:
us_vcp_hourly_countNA = us_vcp_hourly_df.isna().sum()
us_vcp_hourly_countNA

TIMESTAMP_START        0
TIMESTAMP_END          0
TA_F                   0
TA_F_QC                0
TA_ERA                 0
                   ...  
b6                 17664
b7                 10896
IGBP                   0
koppen                 0
min_categorical        0
Length: 65, dtype: int64

In [19]:
us_vcp_hourly_countNA = pd.DataFrame(us_vcp_hourly_df.isna().sum())

NA0_list = list(us_vcp_hourly_countNA[us_vcp_hourly_countNA[0] == 0].index)
NA_list = list(us_vcp_hourly_countNA[us_vcp_hourly_countNA[0] != 0].index)

In [20]:
us_vcp_hourly_countNA[us_vcp_hourly_countNA[0] != 0]

,0
NETRAD,50346
PPFD_IN,13861
G_F_MDS,174528
G_F_MDS_QC,174528
LE_F_MDS_QC,174528
LE_CORR,174528
H_F_MDS_QC,174528
H_CORR,174528
NEE_VUT_REF_qa,7628
SW_DIF,174528


In [28]:
# Features used for this analysis
output_var = 'GPP_NT_VUT_REF'
# other options
output_related_var = ['NEE_VUT_REF', 'NEE_VUT_REF_QC',
       'NEE_CUT_REF', 'NEE_CUT_REF_QC', 'GPP_NT_VUT_REF', 'GPP_DT_VUT_REF',
       'GPP_NT_CUT_REF', 'GPP_DT_CUT_REF', 'RECO_NT_VUT_REF',
       'RECO_DT_VUT_REF', 'RECO_NT_CUT_REF', 'RECO_DT_CUT_REF',
       'NEE_VUT_REF_qa']

pred_var_numeric = ['TA_F', 'TA_ERA','SW_IN_POT', 'SW_IN_F','SW_IN_ERA',
                 'LW_IN_F','LW_IN_ERA', 'VPD_F', 'VPD_ERA',
                 'P_F','P_ERA', 'PA_F',  'PA_ERA', 'NETRAD', 'PPFD_IN',
                 'LE_F_MDS', 'H_F_MDS', 'EVI', 'NDVI', 'NIRv',
                 'b1', 'b2', 'b3', 'b4', 'b5', 'b6','b7',
                 'year', 'month', 'day', 'hour' ]

pred_var_categorical = ['koppen']

other_var =  ['TIMESTAMP_START', 'TIMESTAMP_END', 'datetime',
              'SITE_ID', 'date', 'IGBP',
              'koppen']

qc_flags = ['TA_F_QC','SW_IN_F_QC','LW_IN_F_QC','VPD_F_QC',
            'P_F_QC', 'PA_F_QC','G_F_MDS_QC', 'LE_F_MDS_QC',
            'H_F_MDS_QC'] 

NA_list = ['G_F_MDS', 'LE_CORR', 'H_CORR','SW_DIF']
           # 'G_F_MDS_QC', 'LE_F_MDS_QC', 'H_F_MDS_QC' is skipped because they are included in qc flags
len(output_related_var + pred_var_numeric + pred_var_categorical + other_var + qc_flags + NA_list)
# output_var is included in output_related_var

65

In [29]:
# Check if all the features are included in one of the four
total = output_related_var + pred_var_numeric + pred_var_categorical + other_var + qc_flags + NA_list
for i in us_vcp_hourly_df.columns:
  if i not in total:
    print(i)

min_categorical


### Predictor variables(candidates)

| feature name |     source    |                                                              definition                                                              |
|:------------:|:-------------:|:------------------------------------------------------------------------------------------------------------------------------------:|
| TA_F         |    FLUXNET    | Air temperature, consolidated from TA_F_MDS and TA_ERA                                                                               |
| TA_ERA       |    FLUXNET    | Air temperature, downscaled from ERA, linearly regressed using measured only site data                                               |
| SW_IN_POT    |    FLUXNET    | Shortwave radiation, incoming, potential (top of atmosphere)                                                                         |
| SW_IN_F      |    FLUXNET    | Shortwave radiation,  incoming consolidated from SW_IN_F_MDS and SW_IN_ERA (negative values set to zero)                             |
| SW_IN_ERA    |    FLUXNET    | Shortwave radiation, incoming, downscaled from ERA,  linearly regressed using measured only site data  (negative values set to zero) |
| LW_IN_F      |    FLUXNET    | Longwave radiation, incoming, consolidated from LW_IN_F_MDS and LW_IN_ERA                                                            |
| LW_IN_ERA    |    FLUXNET    | Longwave radiation, incoming, downscaled from ERA, linearly regressed using measured only site data                                  |
| VPD_F        |    FLUXNET    | N/A                                                                                                                                  |
| VPD_ERA      |    FLUXNET    | Vapor Pressure Deficit, downscaled from ERA, linearly regressed using measured only site data                                        |
| P_F          |    FLUXNET    | Precipitation consolidated from P and P_ERA                                                                                          |
| P_ERA        |    FLUXNET    | Precipitation, downscaled from ERA, linearly regressed using measured only site data                                                 |
| PA_F         |    FLUXNET    | Atmospheric pressure consolidated from PA and PA_ERA                                                                                 |
| PA_ERA       |    FLUXNET    | Atmospheric pressure, downscaled from ERA, linearly regressed using measured only site data                                          |
| NETRAD       |    FLUXNET    | Net radiation                                                                                                                        |
| PPFD_IN      |    FLUXNET    | Photosynthetic photon flux density, incoming                                                                                         |
| G_F_MDS      |    FLUXNET    | Soil heat flux                                                                                                                       |
| LE_F_MDS     |    FLUXNET    | Latent heat flux, gapfilled using MDS method                                                                                         |
| LE_CORR      |    FLUXNET    | Latent heat flux, corrected LE_F_MDS by energy balance closure correction factor                                                     |
| H_F_MDS      |    FLUXNET    | Sensible heat flux, gapfilled using MDS method                                                                                       |
| H_CORR       |    FLUXNET    | Sensible heat flux, corrected H_F_MDS by energy balance closure correction factor                                                    |
| SW_DIF       |    FLUXNET    | Shortwave radiation, diffuse incoming                                                                                                |
| EVI          |    MCD43C4    | Enhanced Vegetation Index (EVI)                                                                                                      |
| NDVI         |    MCD43C4    | Normalized Difference Vegetation Index (NDVI)                                                                                        |
| NIRv         |    MCD43C4    | NIRv d                                                                                                                               |
| b1           |    MCD43C4    | Surface reflectance Band 1                                                                                                           |
| b2           |    MCD43C4    | Surface reflectance Band 2 (nir)                                                                                                     |
| b3           |    MCD43C4    | Surface reflectance Band 3 (blue)                                                                                                    |
| b4           |    MCD43C4    | Surface reflectance Band 4 (green)                                                                                                   |
| b5           |    MCD43C4    | Surface reflectance Band 5 (SWIR1)                                                                                                   |
| b6           |    MCD43C4    | Surface reflectance Band 6 (SWIR2)                                                                                                   |
| b7           |    MCD43C4    | Surface reflectance Band 7 (SWIR3)                                                                                                   |
| koppen       | Koppen-Geiger | Climate zone (one-hot encoding)                                                                                                      |

---
*Following process use only Spark(N0 PANDAS)*

# Use parquet -> Spark df as preparation for global data

## DF to Parquet format
- Pandas is converted to parquet in order to make the following code usable with global data in AWS

In [30]:
type(us_vcp_hourly_00_df)

pandas.core.frame.DataFrame

### Method 1

In [31]:
!pip install fastparquet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 58.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 79.5 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5


In [32]:
us_vcp_hourly_00_df.to_parquet('us_vcp_hourly_00_pq', engine='fastparquet')
us_vcp_hourly_df.to_parquet('us_vcp_hourly_pq', engine='fastparquet')

### Method 2

In [33]:
# Convert df to parquet
import pyarrow as pa
import pyarrow.parquet as pq

# Convert DataFrame to Apache Arrow Table
table = pa.Table.from_pandas(us_vcp_hourly_00_df)
# Second, write the table into parquet file say file_name.parquet
# Parquet with Brotli compression
pq.write_table(table, 'us_vcp_hourly_00_pq.parquet')

## parquet to Spark df 

In [34]:
# Parquet is read from 
# When running Spark in AWS through the access to Azure, update the location and file name 
us_vcp_hourly_00_sdf = spark.read.parquet('us_vcp_hourly_00_pq')
us_vcp_hourly_00_sdf2 = spark.read.parquet('us_vcp_hourly_00_pq.parquet')
us_vcp_hourly_sdf = spark.read.parquet('us_vcp_hourly_pq')

In [35]:
us_vcp_hourly_00_sdf.printSchema()

root
 |-- index: long (nullable = true)
 |-- TIMESTAMP_START: long (nullable = true)
 |-- TIMESTAMP_END: long (nullable = true)
 |-- TA_F: double (nullable = true)
 |-- TA_F_QC: long (nullable = true)
 |-- TA_ERA: double (nullable = true)
 |-- SW_IN_POT: double (nullable = true)
 |-- SW_IN_F: double (nullable = true)
 |-- SW_IN_F_QC: long (nullable = true)
 |-- SW_IN_ERA: double (nullable = true)
 |-- LW_IN_F: double (nullable = true)
 |-- LW_IN_F_QC: double (nullable = true)
 |-- LW_IN_ERA: double (nullable = true)
 |-- VPD_F: double (nullable = true)
 |-- VPD_F_QC: long (nullable = true)
 |-- VPD_ERA: double (nullable = true)
 |-- P_F: double (nullable = true)
 |-- P_F_QC: double (nullable = true)
 |-- P_ERA: double (nullable = true)
 |-- PA_F: double (nullable = true)
 |-- PA_F_QC: double (nullable = true)
 |-- PA_ERA: double (nullable = true)
 |-- NETRAD: double (nullable = true)
 |-- PPFD_IN: double (nullable = true)
 |-- G_F_MDS: double (nullable = true)
 |-- G_F_MDS_QC: double (

In [36]:
us_vcp_hourly_00_sdf.show(truncate=False)

+-----+---------------+-------------+------+-------+-------+---------+-------+----------+---------+-------+----------+---------+-----+--------+-------+---+------+-----+------+-------+------+----------+-------+-------+----------+---------+-----------+-------+--------+----------+------+-----------+--------------+-----------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+---------------+---------------+-------------------+----+-----+---+----+-------+----------+--------------+------+------------------+------------------+------------------+------+------+-----+------+------+------+------+----+------+---------------+
|index|TIMESTAMP_START|TIMESTAMP_END|TA_F  |TA_F_QC|TA_ERA |SW_IN_POT|SW_IN_F|SW_IN_F_QC|SW_IN_ERA|LW_IN_F|LW_IN_F_QC|LW_IN_ERA|VPD_F|VPD_F_QC|VPD_ERA|P_F|P_F_QC|P_ERA|PA_F  |PA_F_QC|PA_ERA|NETRAD    |PPFD_IN|G_F_MDS|G_F_MDS_QC|LE_F_MDS |LE_F_MDS_QC|LE_CORR|H_F_MDS |H_F_MDS_QC|H_CORR|NEE_VUT_REF|NEE_VUT_REF_QC|NEE_CUT_RE

In [37]:
display(us_vcp_hourly_00_sdf)

DataFrame[index: bigint, TIMESTAMP_START: bigint, TIMESTAMP_END: bigint, TA_F: double, TA_F_QC: bigint, TA_ERA: double, SW_IN_POT: double, SW_IN_F: double, SW_IN_F_QC: bigint, SW_IN_ERA: double, LW_IN_F: double, LW_IN_F_QC: double, LW_IN_ERA: double, VPD_F: double, VPD_F_QC: bigint, VPD_ERA: double, P_F: double, P_F_QC: double, P_ERA: double, PA_F: double, PA_F_QC: double, PA_ERA: double, NETRAD: double, PPFD_IN: double, G_F_MDS: double, G_F_MDS_QC: double, LE_F_MDS: double, LE_F_MDS_QC: double, LE_CORR: double, H_F_MDS: double, H_F_MDS_QC: double, H_CORR: double, NEE_VUT_REF: double, NEE_VUT_REF_QC: double, NEE_CUT_REF: double, NEE_CUT_REF_QC: double, GPP_NT_VUT_REF: double, GPP_DT_VUT_REF: double, GPP_NT_CUT_REF: double, GPP_DT_CUT_REF: double, RECO_NT_VUT_REF: double, RECO_DT_VUT_REF: double, RECO_NT_CUT_REF: double, RECO_DT_CUT_REF: double, datetime: string, year: bigint, month: bigint, day: bigint, hour: bigint, SITE_ID: string, date: string, NEE_VUT_REF_qa: double, SW_DIF: double

In [38]:
us_vcp_hourly_sdf.show()

+---------------+-------------+------+-------+-------+---------+-------+----------+---------+-------+----------+---------+-----+--------+-------+---+------+-----+------+-------+------+----------+-------+-------+----------+---------+-----------+-------+--------+----------+------+-----------+--------------+-----------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+---------------+---------------+-------------------+----+-----+---+----+-------+----------+--------------+------+------------------+------------------+------------------+------+------+-----+------+------+------+------+----+------+---------------+
|TIMESTAMP_START|TIMESTAMP_END|  TA_F|TA_F_QC| TA_ERA|SW_IN_POT|SW_IN_F|SW_IN_F_QC|SW_IN_ERA|LW_IN_F|LW_IN_F_QC|LW_IN_ERA|VPD_F|VPD_F_QC|VPD_ERA|P_F|P_F_QC|P_ERA|  PA_F|PA_F_QC|PA_ERA|    NETRAD|PPFD_IN|G_F_MDS|G_F_MDS_QC| LE_F_MDS|LE_F_MDS_QC|LE_CORR| H_F_MDS|H_F_MDS_QC|H_CORR|NEE_VUT_REF|NEE_VUT_REF_QC|NEE_CUT_REF|NEE_CUT_RE

In [39]:
# size of two datasets
print(f'us_vcp_hourly_00_sdf: {us_vcp_hourly_00_sdf.count()}')
print(f'us_vcp_hourly_sdf: {us_vcp_hourly_sdf.count()}')

us_vcp_hourly_00_sdf: 87264
us_vcp_hourly_sdf: 174528


In [40]:
print(type(us_vcp_hourly_00_sdf))
print(type(us_vcp_hourly_sdf))

<class 'pyspark.sql.dataframe.DataFrame'>
<class 'pyspark.sql.dataframe.DataFrame'>


In [41]:
us_vcp_hourly_sdf.columns

['TIMESTAMP_START',
 'TIMESTAMP_END',
 'TA_F',
 'TA_F_QC',
 'TA_ERA',
 'SW_IN_POT',
 'SW_IN_F',
 'SW_IN_F_QC',
 'SW_IN_ERA',
 'LW_IN_F',
 'LW_IN_F_QC',
 'LW_IN_ERA',
 'VPD_F',
 'VPD_F_QC',
 'VPD_ERA',
 'P_F',
 'P_F_QC',
 'P_ERA',
 'PA_F',
 'PA_F_QC',
 'PA_ERA',
 'NETRAD',
 'PPFD_IN',
 'G_F_MDS',
 'G_F_MDS_QC',
 'LE_F_MDS',
 'LE_F_MDS_QC',
 'LE_CORR',
 'H_F_MDS',
 'H_F_MDS_QC',
 'H_CORR',
 'NEE_VUT_REF',
 'NEE_VUT_REF_QC',
 'NEE_CUT_REF',
 'NEE_CUT_REF_QC',
 'GPP_NT_VUT_REF',
 'GPP_DT_VUT_REF',
 'GPP_NT_CUT_REF',
 'GPP_DT_CUT_REF',
 'RECO_NT_VUT_REF',
 'RECO_DT_VUT_REF',
 'RECO_NT_CUT_REF',
 'RECO_DT_CUT_REF',
 'datetime',
 'year',
 'month',
 'day',
 'hour',
 'SITE_ID',
 'date',
 'NEE_VUT_REF_qa',
 'SW_DIF',
 'EVI',
 'NDVI',
 'NIRv',
 'b1',
 'b2',
 'b3',
 'b4',
 'b5',
 'b6',
 'b7',
 'IGBP',
 'koppen',
 'min_categorical']

In [42]:
# Features used for this analysis
output_var = 'GPP_NT_VUT_REF'
# other options
output_related_var = ['NEE_VUT_REF', 'NEE_VUT_REF_QC',
       'NEE_CUT_REF', 'NEE_CUT_REF_QC', 'GPP_NT_VUT_REF', 'GPP_DT_VUT_REF',
       'GPP_NT_CUT_REF', 'GPP_DT_CUT_REF', 'RECO_NT_VUT_REF',
       'RECO_DT_VUT_REF', 'RECO_NT_CUT_REF', 'RECO_DT_CUT_REF',
       'NEE_VUT_REF_qa']

pred_var_numeric = ['TA_F', 'TA_ERA','SW_IN_POT', 'SW_IN_F','SW_IN_ERA',
                 'LW_IN_F','LW_IN_ERA', 'VPD_F', 'VPD_ERA',
                 'P_F','P_ERA', 'PA_F',  'PA_ERA', 'NETRAD', 'PPFD_IN',
                 'LE_F_MDS', 'H_F_MDS', 'EVI', 'NDVI', 'NIRv',
                 'b1', 'b2', 'b3', 'b4', 'b5', 'b6','b7',
                 'year', 'month', 'day', 'hour' ]

pred_var_categorical = ['koppen']

other_var =  ['TIMESTAMP_START', 'TIMESTAMP_END', 'datetime',
              'SITE_ID', 'date', 'IGBP',
              'koppen']

qc_flags = ['TA_F_QC','SW_IN_F_QC','LW_IN_F_QC','VPD_F_QC',
            'P_F_QC', 'PA_F_QC','G_F_MDS_QC', 'LE_F_MDS_QC',
            'H_F_MDS_QC'] 

NA_list = ['G_F_MDS', 'LE_CORR', 'H_CORR','SW_DIF']
           # 'G_F_MDS_QC', 'LE_F_MDS_QC', 'H_F_MDS_QC' is skipped because they are included in qc flags
len(output_related_var + pred_var_numeric + pred_var_categorical + other_var + qc_flags + NA_list)
# output_var is included in output_related_var

64

In [43]:
# Features for RFRegressor 
feature_list = pred_var_numeric + pred_var_categorical # [output_var] is not included in the list

In [44]:
# Update Spark dataframe
us_vcp_hourly_rf = us_vcp_hourly_sdf.select([output_var] + feature_list)
us_vcp_hourly_rf.show()

+--------------+------+-------+---------+-------+---------+-------+---------+-----+-------+---+-----+------+------+----------+-------+---------+--------+------------------+------------------+------------------+------+------+-----+------+------+------+------+----+-----+---+----+------+
|GPP_NT_VUT_REF|  TA_F| TA_ERA|SW_IN_POT|SW_IN_F|SW_IN_ERA|LW_IN_F|LW_IN_ERA|VPD_F|VPD_ERA|P_F|P_ERA|  PA_F|PA_ERA|    NETRAD|PPFD_IN| LE_F_MDS| H_F_MDS|               EVI|              NDVI|              NIRv|    b1|    b2|   b3|    b4|    b5|    b6|    b7|year|month|day|hour|koppen|
+--------------+------+-------+---------+-------+---------+-------+---------+-----+-------+---+-----+------+------+----------+-------+---------+--------+------------------+------------------+------------------+------+------+-----+------+------+------+------+----+-----+---+----+------+
|     -0.178472|-7.706| -9.299|      0.0|    0.0|      0.0|202.327|  202.327|0.484|   1.58|0.0|  0.0|75.633|75.452|-84.713875|   null| 0.87444

NETRAD has large amount of missing values

In [45]:
# counts the number of None in each feature
from pyspark.sql.functions import isnull, when, count, col
us_vcp_hourly_rf.select([count(when(isnull(c), c)).alias(c) for c in us_vcp_hourly_rf.columns]).show()

+--------------+----+------+---------+-------+---------+-------+---------+-----+-------+---+-----+----+------+------+-------+--------+-------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+----+-----+---+----+------+
|GPP_NT_VUT_REF|TA_F|TA_ERA|SW_IN_POT|SW_IN_F|SW_IN_ERA|LW_IN_F|LW_IN_ERA|VPD_F|VPD_ERA|P_F|P_ERA|PA_F|PA_ERA|NETRAD|PPFD_IN|LE_F_MDS|H_F_MDS|  EVI| NDVI| NIRv|   b1|   b2|   b3|   b4|   b5|   b6|   b7|year|month|day|hour|koppen|
+--------------+----+------+---------+-------+---------+-------+---------+-----+-------+---+-----+----+------+------+-------+--------+-------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+----+-----+---+----+------+
|             0|   0|     0|        0|      0|        0|      0|        0|    0|      0|  0|    0|   0|     0| 50346|  13861|       0|      0|10992|10944|10944|10944|10896|10992|10944|12240|17664|10896|   0|    0|  0|   0|     0|
+--------------+----+------+---------+-------+---------+-------+---------+-----+

In [46]:
rf_noNETRAD_list = ['TA_F','TA_ERA', 'SW_IN_POT', 'SW_IN_F',
                    'SW_IN_ERA', 'LW_IN_F', 'LW_IN_ERA','VPD_F','VPD_ERA',
                    'P_F', 'P_ERA', 'PA_F', 'PA_ERA', 'PPFD_IN',
                    'LE_F_MDS', 'H_F_MDS', 'EVI', 'NDVI', 'NIRv',
                    'b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7',
                    'year', 'month', 'day', 'hour'] + pred_var_categorical#'NETRAD',
output_var = 'GPP_NT_VUT_REF'

In [47]:
# Remove NETRAD from column and count the rows
us_vcp_hourly_rf_NAdrop_noNETRAD = us_vcp_hourly_rf.select([output_var] + rf_noNETRAD_list)
us_vcp_hourly_rf_NAdrop_noNETRAD = us_vcp_hourly_rf_NAdrop_noNETRAD.na.drop("any")
us_vcp_hourly_rf_NAdrop_noNETRAD.count()

143255

In [48]:
# Keep NETRAD from column and count the rows
us_vcp_hourly_rf_NAdrop = us_vcp_hourly_rf.na.drop("any")
us_vcp_hourly_rf_NAdrop.count()

104846

Use `us_vcp_hourly_rf_NAdrop` in the later phase
- Assuming NETRAD is also significant feature, use the df with NETRAD first.
- Based on the result obtained from smaller df, decide whether to drop NETRAD and use the larger dataset

In [49]:
us_vcp_hourly_rf_NAdrop.show()

+--------------+------+------+---------+-------+---------+-------+---------+-----+-------+---+-----+------+------+----------+---------+--------+--------+------------------+------------------+------------------+------+------+------+------+------+------+------+----+-----+---+----+------+
|GPP_NT_VUT_REF|  TA_F|TA_ERA|SW_IN_POT|SW_IN_F|SW_IN_ERA|LW_IN_F|LW_IN_ERA|VPD_F|VPD_ERA|P_F|P_ERA|  PA_F|PA_ERA|    NETRAD|  PPFD_IN|LE_F_MDS| H_F_MDS|               EVI|              NDVI|              NIRv|    b1|    b2|    b3|    b4|    b5|    b6|    b7|year|month|day|hour|koppen|
+--------------+------+------+---------+-------+---------+-------+---------+-----+-------+---+-----+------+------+----------+---------+--------+--------+------------------+------------------+------------------+------+------+------+------+------+------+------+----+-----+---+----+------+
|      -3.47747|11.853|19.185|      0.0|    0.0|      0.0|316.061|  316.061|1.913| 14.306|0.0|  0.0|75.885| 75.46|-41.358245|      0.0| 5.2

# RF feature selection with pyspark
- In modeling, train-test split(+cross validation) needs to be implemented additionally

In [50]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [51]:
from pyspark.mllib.stat import Statistics
from pyspark.ml.stat import Correlation
from pyspark.sql.functions import col

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import MinMaxScaler
from pyspark.mllib.evaluation import MulticlassMetrics, BinaryClassificationMetrics
from pyspark.ml.feature import StringIndexer, OneHotEncoder

# Import fro classifier
# from pyspark.ml.evaluation import BinaryClassificationEvaluator
# from pyspark.ml.classification import RandomForestClassifier

from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

# Documentation for RFRegressor in Spark
# https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.regression.RandomForestRegressor.html

## Convert categorical variable to numeric
- For monthly dataset, we need to go through the steps because of the several categorical variables

### StringIndexer

- Convert string categorical variables to numeric variables

In [52]:
# Categorical features in monthly dataset
pred_var_categorical

['koppen']

In [58]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder
 
categoricalCols = pred_var_categorical
 
# The following two lines are estimators. They return functions that we will later apply to transform the dataset.
stringIndexer = StringIndexer(inputCols=categoricalCols, outputCols=[x + "_Index" for x in categoricalCols]) 
encoder = OneHotEncoder(inputCols=stringIndexer.getOutputCols(), outputCols=[x + "_OHE" for x in categoricalCols]) 
 
# Example
# If the label column ("income") is also a string value - let's say it has two possible values, "<=50K" and ">50K". 
# Convert it to a numeric value using StringIndexer.
# labelToIndex = StringIndexer(inputCol="income", outputCol="label")

For Categorical variables, new column XXX_Index is added and numerical values are assigned to each category.

In [59]:
# This case we run code for feature importance, not modeling /prediction 
# Therefore full dataset is provided as input
stringIndexerModel = stringIndexer.fit(us_vcp_hourly_rf_NAdrop)
us_vcp_hourly_rf_NAdrop_STI = stringIndexerModel.transform(us_vcp_hourly_rf_NAdrop) # STI = StringIndexer

# Typically input data is train set/holdout set
    # stringIndexerModel = stringIndexer.fit(train0)
    # train0_STI = stringIndexerModel.transform(train0) # STI = StringIndexer
    # display(train0_STI)

  # same operation for holdout data
    # stringIndexerModel = stringIndexer.fit(holdout0)
    # holdout0_STI = stringIndexerModel.transform(holdout0) # STI = StringIndexer
    # display(holdout0_STI)

In [61]:
us_vcp_hourly_rf_NAdrop_STI.show()

+--------------+------+------+---------+-------+---------+-------+---------+-----+-------+---+-----+------+------+----------+---------+--------+--------+------------------+------------------+------------------+------+------+------+------+------+------+------+----+-----+---+----+------+------------+
|GPP_NT_VUT_REF|  TA_F|TA_ERA|SW_IN_POT|SW_IN_F|SW_IN_ERA|LW_IN_F|LW_IN_ERA|VPD_F|VPD_ERA|P_F|P_ERA|  PA_F|PA_ERA|    NETRAD|  PPFD_IN|LE_F_MDS| H_F_MDS|               EVI|              NDVI|              NIRv|    b1|    b2|    b3|    b4|    b5|    b6|    b7|year|month|day|hour|koppen|koppen_Index|
+--------------+------+------+---------+-------+---------+-------+---------+-----+-------+---+-----+------+------+----------+---------+--------+--------+------------------+------------------+------------------+------+------+------+------+------+------+------+----+-----+---+----+------+------------+
|      -3.47747|11.853|19.185|      0.0|    0.0|      0.0|316.061|  316.061|1.913| 14.306|0.0|  0.0|

### OneHotEncoder

Transform the string values by `stringIndexer.fit` `stringIndexer.transform` first, and then use OneHotEncoder.

This link is helpful to understand the structure of column generated by `OneHotEncoder`
https://www.skytowner.com/explore/one_hot_encoding_in_pyspark

In [63]:

# 'MODIS_LC_Index' -> 'MODIS_LC_OHE' 
# 'MODIS_IGBP_Index' -> 'MODIS_IGBP_OHE' 
# 'MODIS_PFT_Index' -> 'MODIS_PFT_OHE'
# 'koppen_sub_Index' -> 'koppen_sub_OHE'
# 'koppen_Index' -> 'koppen_OHE'

# OHE = OneHotEncoding
encoder = OneHotEncoder(inputCols=['koppen_Index'],
                        outputCols=['koppen_OHE'])
model = encoder.fit(us_vcp_hourly_rf_NAdrop_STI)
monthly_rf_NAdrop_OHE = model.transform(us_vcp_hourly_rf_NAdrop_STI)
monthly_rf_NAdrop_OHE.show()

# model = encoder.fit(holdout0_STI)
# holdout0_OHE = model.transform(holdout0_STI)
# display(holdout0_OHE)

IllegalArgumentException: ignored

 => koppen did not have any distinct value, therefore it is removed from feature candidates

## Assemble features

### Update feature list(If categorical vairable exists)

In [64]:
# New feature list with OHE variables

# Keep original list for numeric features
pred_var_numeric = ['TA_F','TA_ERA', 'SW_IN_POT', 'SW_IN_F',
                    'SW_IN_ERA', 'LW_IN_F', 'LW_IN_ERA','VPD_F','VPD_ERA',
                    'P_F', 'P_ERA', 'PA_F', 'PA_ERA', 'NETRAD', 'PPFD_IN',
                    'LE_F_MDS', 'H_F_MDS', 'EVI', 'NDVI', 'NIRv',
                    'b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7',
                    'year', 'month', 'day', 'hour']
output_var = 'GPP_NT_VUT_REF'

# Update variable names of categorical features
# Original: pred_var_categorical = ['SITE_IGBP','MODIS_LC', 'MODIS_IGBP','MODIS_PFT', 'koppen_sub', 'koppen']
pred_var_categorical_OHE = []

feature_list = pred_var_numeric + pred_var_categorical_OHE


In [65]:
# Updated feature list
feature_list

['TA_F',
 'TA_ERA',
 'SW_IN_POT',
 'SW_IN_F',
 'SW_IN_ERA',
 'LW_IN_F',
 'LW_IN_ERA',
 'VPD_F',
 'VPD_ERA',
 'P_F',
 'P_ERA',
 'PA_F',
 'PA_ERA',
 'NETRAD',
 'PPFD_IN',
 'LE_F_MDS',
 'H_F_MDS',
 'EVI',
 'NDVI',
 'NIRv',
 'b1',
 'b2',
 'b3',
 'b4',
 'b5',
 'b6',
 'b7',
 'year',
 'month',
 'day',
 'hour']

In [66]:
# Assemble features
assembler = VectorAssembler().setInputCols(feature_list)\
                            .setOutputCol('vectorized_features')

In [68]:
# Usually the input is train set/ holdout set
# Full dataset is used here because the goal of this process is to apply feture importance
# Example code
  # assmb_train0 = assembler.transform(train0)#train0_OHE)
  # assmb_holdout0 = assembler.transform(holdout0)#_OHE)

assmb_sdf_NAdrop_OHE = assembler.transform(us_vcp_hourly_rf_NAdrop_STI) 
# If there exist any categorical variable, us_vcp_hourly_rf_NAdrop_OHE was used instead


In [69]:
# vectorized features are added to the lest side of table
assmb_sdf_NAdrop_OHE.show()

+--------------+------+------+---------+-------+---------+-------+---------+-----+-------+---+-----+------+------+----------+---------+--------+--------+------------------+------------------+------------------+------+------+------+------+------+------+------+----+-----+---+----+------+------------+--------------------+
|GPP_NT_VUT_REF|  TA_F|TA_ERA|SW_IN_POT|SW_IN_F|SW_IN_ERA|LW_IN_F|LW_IN_ERA|VPD_F|VPD_ERA|P_F|P_ERA|  PA_F|PA_ERA|    NETRAD|  PPFD_IN|LE_F_MDS| H_F_MDS|               EVI|              NDVI|              NIRv|    b1|    b2|    b3|    b4|    b5|    b6|    b7|year|month|day|hour|koppen|koppen_Index| vectorized_features|
+--------------+------+------+---------+-------+---------+-------+---------+-----+-------+---+-----+------+------+----------+---------+--------+--------+------------------+------------------+------------------+------+------+------+------+------+------+------+----+-----+---+----+------+------------+--------------------+
|      -3.47747|11.853|19.185|      0

## Normalize features

In [70]:
# normalize dataset
scaler = MinMaxScaler().setInputCol('vectorized_features')\
                            .setOutputCol('normalized_features')

scaler_sdf_NAdrop_model = scaler.fit(assmb_sdf_NAdrop_OHE)
normed_sdf_NAdrop = scaler_sdf_NAdrop_model.transform(assmb_sdf_NAdrop_OHE)

# Same as the last section, usually input is train set/holdout set
# Example
    # normed_train0 = scaler_model0.transform(assmb_train0)
    # normed_holdout0 = scaler_model0.transform(assmb_holdout0)

In [71]:
# Column 'normalized features' is added
normed_sdf_NAdrop.show()


+--------------+------+------+---------+-------+---------+-------+---------+-----+-------+---+-----+------+------+----------+---------+--------+--------+------------------+------------------+------------------+------+------+------+------+------+------+------+----+-----+---+----+------+------------+--------------------+--------------------+
|GPP_NT_VUT_REF|  TA_F|TA_ERA|SW_IN_POT|SW_IN_F|SW_IN_ERA|LW_IN_F|LW_IN_ERA|VPD_F|VPD_ERA|P_F|P_ERA|  PA_F|PA_ERA|    NETRAD|  PPFD_IN|LE_F_MDS| H_F_MDS|               EVI|              NDVI|              NIRv|    b1|    b2|    b3|    b4|    b5|    b6|    b7|year|month|day|hour|koppen|koppen_Index| vectorized_features| normalized_features|
+--------------+------+------+---------+-------+---------+-------+---------+-----+-------+---+-----+------+------+----------+---------+--------+--------+------------------+------------------+------------------+------+------+------+------+------+------+------+----+-----+---+----+------+------------+-----------------

## Fit model

### Random Forest Regressor

In [75]:
from pyspark.ml.regression import RandomForestRegressor

rf = RandomForestRegressor(featuresCol = 'normalized_features', labelCol = 'GPP_NT_VUT_REF')
rfModel = rf.fit(normed_sdf_NAdrop)
# rfModel = rf.fit(normed_train0)
# predictions = rfModel.transform(normed_holdout0)

## Feature Importance

In [76]:
feature_importance = rfModel.featureImportances

In [77]:
feature_importances_list = [feature_list]
feature_importances_list += [list(feature_importance)]
feature_importances_df = pd.DataFrame(feature_importances_list).T

In [78]:
feature_importances_df.columns = ["feature_name", "importance"]
feature_importances_df


,feature_name,importance
0,TA_F,0.014083
1,TA_ERA,0.016697
2,SW_IN_POT,0.042149
3,SW_IN_F,0.136793
4,SW_IN_ERA,0.012494
5,LW_IN_F,0.000063
6,LW_IN_ERA,0.000173
7,VPD_F,0.012122
8,VPD_ERA,0.011401
9,P_F,0.0


In [79]:
!pip install pandas==1.3.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.6/10.6 MB 34.2 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastparquet 2023.1.0 requires pandas>=1.5.0, but you have pandas 1.3.0 which is incompatible.


In [80]:
feature_importances_df.to_csv("feature_importances_df_hourly_US-vcp.csv")